## Collaborative Filtering

### K Nearest Neighbors

In [1]:
import pandas as pd
import numpy as np
books = pd.read_csv("popular_books.csv")
ratings = pd.read_csv("Ratings.csv", low_memory=False)

In [2]:
print(books.shape)
print(list(books.columns))

(1913, 8)
['ISBN', 'AvgBookRating', 'NumberOfRating', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher', 'new_score']


In [3]:
ratings = ratings.rename(columns={"User-ID": "UserID", "Book-Rating":"BookRating"})
print(ratings.shape)
print(list(ratings.columns))

(1149780, 3)
['UserID', 'ISBN', 'BookRating']


In [4]:
list_of_books = books['ISBN'].tolist()

In [5]:
list_of_books

['1551668300',
 '0451524934',
 '0446610038',
 '0345303067',
 '1551669293',
 '0451203593',
 '0316693200',
 '0316603570',
 '0451203895',
 '0860074382',
 '1558743316',
 '0743224574',
 '0446611867',
 '0451183665',
 '1558743669',
 '0679772677',
 '0393312836',
 '0345452569',
 '0345452550',
 '0517122707',
 '0802130208',
 '0671886665',
 '0345425707',
 '0440217490',
 '0446667900',
 '0670896764',
 '140003065X',
 '0553573403',
 '0767915054',
 '0553278029',
 '0684863472',
 '0375725784',
 '0553279912',
 '0345423402',
 '0375702709',
 '0140254544',
 '0451188489',
 '0452281903',
 '0374270325',
 '0385720106',
 '0140270590',
 '0553278274',
 '044023722X',
 '0156711427',
 '080411918X',
 '0449003981',
 '0440168724',
 '0449223345',
 '0449225151',
 '0312979533',
 '0345361792',
 '0671776975',
 '0671793535',
 '0553280414',
 '0375504613',
 '0312952716',
 '0345389964',
 '0345347536',
 '0440901588',
 '0061031992',
 '0061000043',
 '0553571885',
 '0449907481',
 '0440211727',
 '0553582364',
 '006092988X',
 '03757030

In [6]:
len(list_of_books)

1913

In [7]:
ratings = ratings[ratings['ISBN'].isin(list_of_books)]

In [8]:
ratings

,UserID,ISBN,BookRating
0,276725,034545104X,0
11,276746,0449006522,0
12,276746,0553561618,0
13,276746,055356451X,0
16,276747,0060517794,9
...,...,...,...
1149761,276704,0345386108,6
1149767,276704,0446353957,0
1149768,276704,0446605409,0
1149771,276704,0743211383,7


In [9]:
counts1 = ratings['UserID'].value_counts()
ratings = ratings[ratings['UserID'].isin(counts1[counts1 >= 50].index)]

In [10]:
ratings

,UserID,ISBN,BookRating
1456,277427,002542730X,10
1468,277427,006092988X,0
1469,277427,0060930535,0
1470,277427,0060932139,0
1471,277427,0060934417,0
...,...,...,...
1147440,275970,1400031354,0
1147441,275970,1400031362,0
1147470,275970,1558744606,0
1147517,275970,1573229725,0


In [11]:
books_details = pd.merge(ratings, books, on=["ISBN"])

In [12]:
books_details

,UserID,ISBN,BookRating,AvgBookRating,NumberOfRating,BookTitle,BookAuthor,YearOfPublication,Publisher,new_score
0,277427,002542730X,10,3.514620,171,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,1.767959
1,3363,002542730X,0,3.514620,171,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,1.767959
2,11676,002542730X,6,3.514620,171,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,1.767959
3,12538,002542730X,10,3.514620,171,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,1.767959
4,13552,002542730X,0,3.514620,171,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,1.767959
...,...,...,...,...,...,...,...,...,...,...
80274,234828,0345333926,8,3.915254,59,Ringworld,Larry Niven,1990,Del Rey Books,1.772682
80275,236283,0345333926,0,3.915254,59,Ringworld,Larry Niven,1990,Del Rey Books,1.772682
80276,249628,0345333926,0,3.915254,59,Ringworld,Larry Niven,1990,Del Rey Books,1.772682
80277,261829,0345333926,0,3.915254,59,Ringworld,Larry Niven,1990,Del Rey Books,1.772682


In [26]:
books_details.to_csv("books_details.csv")

In [13]:
from scipy.sparse import csr_matrix
user_rating_pivot = books_details.pivot(index = 'BookTitle', columns = 'UserID', values = 'BookRating').fillna(0)
user_rating_matrix = csr_matrix(user_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
user_rating_pivot.head(20)

UserID,254,3363,4017,5903,6251,6323,6543,6563,6575,7158,...,273086,273113,273979,274004,274061,274301,274308,275970,277427,278418
BookTitle,,,,,,,,,,,,,,,,,,,,,
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3rd Degree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
user_rating_matrix

<1913x698 sparse matrix of type '<class 'numpy.float64'>'
	with 19289 stored elements in Compressed Sparse Row format>

In [28]:
# query_index = np.random.choice(user_rating_pivot.shape[0])
query_index = 5
print(query_index)
distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 4)

5


In [29]:
user_rating_pivot.iloc[query_index,:].values.reshape(1,-1)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [30]:
user_rating_pivot.index[query_index]

'24 Hours'

In [31]:
recommend_list= []
for i in indices[0]:
    recommend_list.append(user_rating_pivot.index[i])

In [32]:
recommend_list[1:]

['The Perfect Storm: A True Story of Men Against the Sea',
 'The Day After Tomorrow',
 'The Blue Nowhere : A Novel']

In [33]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}'.format(i,user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 24 Hours:

1: The Perfect Storm: A True Story of Men Against the Sea, with distance of 0.5357616545573702
2: The Day After Tomorrow, with distance of 0.7250089370643205
3: The Blue Nowhere : A Novel, with distance of 0.7357509012183834
